# Wildfire risk by US county

## Introduction

### Obtain all zipcodes for the state

In [168]:
state_name = 'CA'
filename = 'data/zipcodes_'+state_name+'.csv'
import pandas as pd
counties = pd.read_csv(filename, names=['zipcode','city','county'])
print("%d zipcodes found..." % len(counties))
with pd.option_context('display.max_rows',10) : 
    print(counties)

2589 zipcodes found...
      zipcode              city       county
0       90001       Los Angeles  Los Angeles
1       90002       Los Angeles  Los Angeles
2       90003       Los Angeles  Los Angeles
3       90004       Los Angeles  Los Angeles
4       90005       Los Angeles  Los Angeles
...       ...               ...          ...
2584    96157  South Lake Tahoe    El Dorado
2585    96158  South Lake Tahoe    El Dorado
2586    96160           Truckee       Nevada
2587    96161           Truckee       Nevada
2588    96162           Truckee       Nevada

[2589 rows x 3 columns]


### Load cencus data 

In [159]:
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)
countydata = {}
skipped_zip=0
for zipcode in list(counties.index) : 
    data = search.by_zipcode(str(zipcode)).to_dict()
    try : 
        countydata[zipcode] = {
            "City":data["major_city"]+" "+data["state"],
            "Latitude":data["lat"],
            "Longitude":data["lng"],
            "Population":data["population"],
            "Density":int(data["population_density"]),
            "Occupancy":int(data["occupied_housing_units"] / data["housing_units"] * 100)/100,
            "Value":int(data["median_home_value"]/100)/10,
            "Income":int(data["median_household_income"]/100)/10
        }
    except:
        skipped_zip = skipped_zip+1
#         print("WARNING: error processing %s rbs, skipping" % (zipcode))
print("WARNING: missing data in %d out of %d zipcodes" % (skipped_zip,len(counties)))

### Load FEMA data

The data is retrieved from https://www.fema.gov/openfema-dataset-disaster-declarations-summaries-v2

In [160]:
fema_filename = 'data/DisasterDeclarationsSummaries.csv'
fire_data = pd.read_csv(fema_filename, index_col=[1],names=["femaDeclarationString",
                                                            "disasterNumber",
                                                            "state",
                                                            "declarationType",
                                                            "declarationDate",
                                                            "fyDeclared",
                                                            "incidentType",
                                                            "declarationTitle",
                                                            "ihProgramDeclared",
                                                            "iaProgramDeclared",
                                                            "paProgramDeclared",
                                                            "hmProgramDeclared",
                                                            "incidentBeginDate",
                                                            "incidentEndDate",
                                                            "disasterCloseoutDate",
                                                            "fipsStateCode",
                                                            "fipsCountyCode",
                                                            "placeCode",
                                                            "designatedArea",
                                                            "declarationRequestNumber",
                                                            "hash","lastRefresh"])
print('Types of incidents filed in the data set...')
print(set(fire_data['incidentType']))
# for item in list(fire_data.index) : 
#     print(item)
# with pd.option_context('display.max_rows',10) : 
#     print(fire_data)

Types of incidents filed in the data set...
{'Flood', 'Human Cause', 'Tsunami', 'Snow', 'Freezing', 'Severe Ice Storm', 'Dam/Levee Break', 'incidentType', 'Tornado', 'Chemical', 'Drought', 'Fire', 'Hurricane', 'Coastal Storm', 'Severe Storm(s)', 'Earthquake', 'Mud/Landslide', 'Fishing Losses', 'Volcano', 'Terrorist', 'Toxic Substances', 'Other', 'Typhoon'}


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,5,8,9,10,11,15,16,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


*Assuming only fire incidents are relevent to this study

### Getting California fire incident data

In [175]:
import requests 
from bs4 import BeautifulSoup
import re
table_content = []
final_list = []
list_of_avail_years = range(2013,2019,1)
for yr in list_of_avail_years : 
    print(yr)
    url = 'https://www.fire.ca.gov/incidents/'+str(yr)+'/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    html_content = {}
    for i in range(len(soup.find_all('div'))) : 
        try : 
            if soup.find_all('div')[i].attrs['class'] == ['section', 'incidents-esri-webmap']: 
                html_content = soup.find_all('div')[i].text
        except : 
            continue
    
    table_content=eval(html_content.partition('create_map([')[2].split("\n")[0][:-3])
    for t in table_content : 
        final_list.append(t)


In [176]:
fires_df = pd.DataFrame(final_list)

Example fire statistics on Ranch Fires

In [181]:
print(fires_df.loc[fires_df['CountiesList'].str.contains('Madera')])

      IncidentId                              UniqueId               Name  \
5            658  357ffc13-bef9-48eb-810f-c5de851972eb          Gold Fire   
14           667  3ddf82b4-0631-4148-807e-181277adeca5        Ranger Fire   
20           673  2b765a08-c770-4b19-baa8-00dffe3fa838       Lilly 2 Fire   
46           699  7055a440-5d4c-4f58-bac9-2954ff6698a5         Revis Fire   
53           706  05d0fed7-8502-4a6f-9c5c-b9cee0e75408       Rolling Fire   
227          880  61c14593-8614-4365-afa7-264fcaa6ea2a        French Fire   
239          892  a2da7754-e907-48f2-a023-0a3053cf2903         Pines Fire   
240          893  58e7e466-8807-404e-9ef6-5a6ae11a0084      Junction Fire   
267          921  614efdbb-937e-47a0-9ffd-919e5e0de436      Courtney Fire   
302          955  3a9bd55c-a384-402b-b673-2563200ce59e           Sky Fire   
303          956  fa8ba2dd-c9f7-454a-870b-1b39c0d8b31a       Corrine Fire   
304          957  0f537c6f-8a74-48f4-9fbf-ad00942ffba0           Red Fire   

Sorting by county by year 

In [164]:
county_list = []
county_list_repeated = set(fires_df['CountiesList'].tolist())
while ("" in county_list_repeated) : 
    county_list_repeated.remove("")
for item in county_list_repeated : 
    item_list = []
    item_list = item.split(",")
    if len(item_list) > 1 : 
        for c in item_list : 
            county_list.append(c.strip())
    else : 
        county_list.append(item)
county_set = set(county_list)
print('List of counties affect by fires : \n', county_set)

List of counties affect by fires : 
 {'Madera', 'El Dorado', 'Sacramento', 'Solano', 'Mariposa', 'Santa Cruz', 'Yolo', 'Mono', 'Yuba', 'Sonoma', 'Alpine', 'Alameda', 'San Diego', 'San Benito', 'Humboldt', 'Sierra', 'Colusa', 'Napa', 'Plumas', 'Santa Barbara', 'Riverside', 'Kings', 'Tulare', 'Los Angeles', 'Placer', 'Santa Clara', 'Amador', 'Fresno', 'Kern', 'Sutter', 'San Joaquin', 'Merced', 'Inyo', 'Tehama', 'Tuolumne', 'Lake', 'Del Norte', 'Siskiyou', 'Shasta', 'Monterey', 'Modoc', 'San Luis Obispo', 'Contra Costa', 'Nevada', 'Mendocino', 'Stanislaus', 'San Bernardino', 'Butte', 'Lassen', 'Trinity', 'San Mateo', 'Glenn', 'Calaveras', 'Orange', 'Marin', 'Ventura'}


In [165]:
print("Total of %d out 58 total counties in CA affected by fires" % len(county_set))

Total of 56 out 58 total counties in CA affected by fires


The county size information was acquired from https://www.counties.org/pod/square-mileage-county on Nov 15, 2019

In [166]:
county_size = pd.read_csv('data/counties_size.csv', names=['county','size'])
print("%d counties found..." % len(county_size))
with pd.option_context('display.max_rows',10) : 
    print(county_size)

58 counties found...
            county   size
0   San Bernardino  20057
1             Inyo  10181
2             Kern   8132
3        Riverside   7206
4         Siskiyou   6278
..             ...    ...
53          Amador    595
54           Marin    520
55       San Mateo    448
56      Santa Cruz    445
57   San Francisco     47

[58 rows x 2 columns]


Note, the area is quoted in square miles.

Checking that the county is listed in the manifest : 

In [167]:
county_manifest = county_size['county'].values.tolist()
for county in county_set : 
    if county not in county_manifest : 
        print("ERROR: Unknown County %s in fire data" % county)

In [171]:
# Summary of required variables
# county_set - counties that had records of fires 
# fires_df - dataframe will all the fire incidents in CA 
# county_size - size of county for multiple counties that were affected by the fire

county_acres_burnt = {}
for county in county_set : 
    acres_burnt = 0
    if county == 
        print(county)

        #     acres_burnt = acres_burnt +
        print(fires_df.loc[fires_df['CountiesList'].str.contains(county),'AcresBurnedDisplay'])
#     print(acres_burnt)

Madera
5          274
14          15
20          91
46          75
53         482
227     13,838
239         10
240        612
267        320
302        500
303        920
304        135
354      5,702
503        103
517         80
538         38
549         85
640        150
683        157
688      1,431
711         25
774         90
912     12,407
931      1,035
961        429
965         48
1082        56
1110       300
1133        60
1164     4,064
1178        12
1205        11
1299       289
1301        88
1323       360
Name: AcresBurnedDisplay, dtype: object
El Dorado
71         572
90         163
102        116
121         38
204         30
223      4,240
264     97,717
270         94
271         20
322         25
323         35
346         75
461         25
487      5,646
533         50
554         25
610        176
763         74
788         13
834        140
842         13
853        236
960        136
1013        29
1021       426
1050        80
1183       316
1269        6